In [2]:
# training_pipeline_debug.ipynb
import pandas as pd
import numpy as np
import boto3
import io
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
from dotenv import load_dotenv

# Załaduj zmienne środowiskowe
load_dotenv()

# Połączenie z DigitalOcean Spaces
session = boto3.session.Session()
s3 = session.client('s3',
                   region_name=os.getenv('DO_SPACES_REGION'),
                   endpoint_url=os.getenv('DO_SPACES_ENDPOINT'),
                   aws_access_key_id=os.getenv('DO_SPACES_KEY'),
                   aws_secret_access_key=os.getenv('DO_SPACES_SECRET'))

# Funkcja do wczytywania CSV z Spaces
def load_csv_from_spaces(filename):
    response = s3.get_object(Bucket=os.getenv('DO_SPACES_BUCKET'), Key=filename)
    return pd.read_csv(io.BytesIO(response['Body'].read()))

print("📥 Wczytywanie danych...")
# Wczytaj dane z 2023 i 2024
df_2023 = load_csv_from_spaces('halfmarathon_wroclaw_2023__final.csv')
df_2024 = load_csv_from_spaces('halfmarathon_wroclaw_2024__final.csv')

print("🔍 STRUKTURA DANYCH 2023:")
print("Kolumny:", df_2023.columns.tolist())
print("Pierwsze 5 wierszy:")
print(df_2023.head())
print("\nInfo:")
print(df_2023.info())

print("\n🔍 STRUKTURA DANYCH 2024:")
print("Kolumny:", df_2024.columns.tolist())
print("Pierwsze 5 wierszy:")
print(df_2024.head())
print("\nInfo:")
print(df_2024.info())

# Sprawdź czy kolumny są takie same
if df_2023.columns.tolist() == df_2024.columns.tolist():
    print("✅ Kolumny są identyczne - można łączyć")
else:
    print("❌ Kolumny są różne - trzeba dostosować")

# Połącz dane jeśli kolumny pasują
df = pd.concat([df_2023, df_2024], ignore_index=True)
print(f"\n📊 Połączone dane: {df.shape}")

# POKAŻ WSZYSTKIE KOLUMNY i ich unikalne wartości
print("\n🎯 ANALIZA KOLUMN:")
for col in df.columns:
    print(f"\n{col}:")
    print(f"  Typ: {df[col].dtype}")
    print(f"  Unikalne wartości: {df[col].nunique()}")
    if df[col].nunique() < 10:
        print(f"  Wartości: {df[col].unique()}")
    print(f"  Braki: {df[col].isnull().sum()}")

📥 Wczytywanie danych...
🔍 STRUKTURA DANYCH 2023:
Kolumny: ['Miejsce;Numer startowy;Imię;Nazwisko;Miasto;Kraj;Drużyna;Płeć;Płeć Miejsce;Kategoria wiekowa;Kategoria wiekowa Miejsce;Rocznik;5 km Czas;5 km Miejsce Open;5 km Tempo;10 km Czas;10 km Miejsce Open;10 km Tempo;15 km Czas;15 km Miejsce Open;15 km Tempo;20 km Czas;20 km Miejsce Open;20 km Tempo;Tempo Stabilność;Czas;Tempo']
Pierwsze 5 wierszy:
  Miejsce;Numer startowy;Imię;Nazwisko;Miasto;Kraj;Drużyna;Płeć;Płeć Miejsce;Kategoria wiekowa;Kategoria wiekowa Miejsce;Rocznik;5 km Czas;5 km Miejsce Open;5 km Tempo;10 km Czas;10 km Miejsce Open;10 km Tempo;15 km Czas;15 km Miejsce Open;15 km Tempo;20 km Czas;20 km Miejsce Open;20 km Tempo;Tempo Stabilność;Czas;Tempo
0  1.0;1787;TOMASZ;GRYCKO;;POL;UKS BLIZA WŁADYSŁA...                                                                                                                                                                                                                                

In [3]:
# training_pipeline_fixed.ipynb
import pandas as pd
import numpy as np
import boto3
import io
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
from dotenv import load_dotenv

# Załaduj zmienne środowiskowe
load_dotenv()

# Połączenie z DigitalOcean Spaces
session = boto3.session.Session()
s3 = session.client('s3',
                   region_name=os.getenv('DO_SPACES_REGION'),
                   endpoint_url=os.getenv('DO_SPACES_ENDPOINT'),
                   aws_access_key_id=os.getenv('DO_SPACES_KEY'),
                   aws_secret_access_key=os.getenv('DO_SPACES_SECRET'))

# Funkcja do wczytywania i rozdzielania CSV
def load_and_split_csv_from_spaces(filename):
    response = s3.get_object(Bucket=os.getenv('DO_SPACES_BUCKET'), Key=filename)
    # Wczytaj jako pojedynczą kolumnę
    df_temp = pd.read_csv(io.BytesIO(response['Body'].read()))
    
    # Rozdziel kolumny według średnika
    df_split = df_temp.iloc[:, 0].str.split(';', expand=True)
    
    # Ustaw nazwy kolumn z pierwszego wiersza
    df_split.columns = df_split.iloc[0]
    df_split = df_split[1:]  # Usuń pierwszy wiersz (nagłówki)
    
    return df_split

print("📥 Wczytywanie i przetwarzanie danych...")
# Wczytaj i rozdziel dane
df_2023 = load_and_split_csv_from_spaces('halfmarathon_wroclaw_2023__final.csv')
df_2024 = load_and_split_csv_from_spaces('halfmarathon_wroclaw_2024__final.csv')

print("🔍 STRUKTURA DANYCH 2023 PO ROZDZIELENIU:")
print("Kolumny:", df_2023.columns.tolist())
print("Wymiar:", df_2023.shape)
print("\nPierwsze 3 wiersze:")
print(df_2023.head(3))

print("\n🔍 STRUKTURA DANYCH 2024 PO ROZDZIELENIU:")
print("Kolumny:", df_2024.columns.tolist())
print("Wymiar:", df_2024.shape)
print("\nPierwsze 3 wiersze:")
print(df_2024.head(3))

# Połącz dane
df = pd.concat([df_2023, df_2024], ignore_index=True)
print(f"\n📊 Połączone dane: {df.shape}")

# ANALIZA KOLUMN - które nas interesują
print("\n🎯 KOLUMNY DO MODELU:")
interesting_cols = ['Płeć', 'Rocznik', '5 km Czas', '5 km Tempo', 'Czas', 'Tempo']
for col in interesting_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Przykładowe wartości: {df[col].head(3).tolist()}")
        print(f"  Unikalne: {df[col].nunique()}")
        print(f"  Braki: {df[col].isnull().sum()}")

# Przygotowanie danych do modelu
print("\n🔧 Przygotowanie danych do trenowania...")

# Konwersja typów danych
df['Rocznik'] = pd.to_numeric(df['Rocznik'], errors='coerce')
df['Czas'] = df['Czas'].astype(str)

# Funkcja do konwersji czasu HH:MM:SS na sekundy
def time_to_seconds(time_str):
    try:
        if pd.isna(time_str):
            return None
        parts = str(time_str).split(':')
        if len(parts) == 3:  # HH:MM:SS
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        elif len(parts) == 2:  # MM:SS
            return int(parts[0]) * 60 + int(parts[1])
        else:
            return None
    except:
        return None

# Konwersja czasów na sekundy
df['half_marathon_seconds'] = df['Czas'].apply(time_to_seconds)
df['pace_5km_seconds'] = df['5 km Tempo'].apply(time_to_seconds)

# Płeć na liczby (M -> 1, K -> 0)
df['sex_numeric'] = df['Płeć'].map({'M': 1, 'K': 0})

# Wiek z rocznika
current_year = 2024
df['age'] = current_year - df['Rocznik']

# Przygotowanie cech i targetu
features = df[['sex_numeric', 'age', 'pace_5km_seconds']].dropna()
target = df.loc[features.index, 'half_marathon_seconds']

print(f"\n📊 Dane do trenowania:")
print(f"Cechy: {features.shape}")
print(f"Target: {target.shape}")

if len(features) > 0:
    # Podział na train/test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    print(f"Train: {X_train.shape}, Test: {X_test.shape}")

    # Trenowanie modelu
    print("\n🤖 Trenowanie modelu...")
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    # Ewaluacja
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"\n📊 Wyniki modelu:")
    print(f"MAE: {mae:.2f} sekund ({mae/60:.1f} minut)")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {np.sqrt(mse):.2f} sekund")

    # Zapis modelu do DigitalOcean Spaces
    print("\n💾 Zapis modelu...")
    model_bytes = io.BytesIO()
    joblib.dump(model, model_bytes)
    model_bytes.seek(0)

    s3.upload_fileobj(model_bytes, 
                      os.getenv('DO_SPACES_BUCKET'), 
                      'trained_model.pkl')

    print("✅ Model zapisany w DO Spaces!")

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': features.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\n🎯 Ważność cech:")
    print(feature_importance)

    print("\n🎉 Pipeline treningowy zakończony!")
else:
    print("❌ Brak danych do trenowania!")

📥 Wczytywanie i przetwarzanie danych...
🔍 STRUKTURA DANYCH 2023 PO ROZDZIELENIU:
Kolumny: ['1.0', '1787', 'TOMASZ', 'GRYCKO', '', 'POL', 'UKS BLIZA WŁADYSŁAWOWO', 'M', '1', 'M30', '1', '1992.0', '00:14:37', '1.0', '2.9233333333333333', '00:29:15', '1.0', '2.9266666666666667', '00:44:47', '1.0', '3.106666666666667', '01:01:43', '1.0', '3.3866666666666663', '0.031399999999999956', '01:04:59', '3.0805088093545074']
Wymiar: (8949, 27)

Pierwsze 3 wiersze:
0  1.0  1787     TOMASZ         GRYCKO           POL  \
1  2.0     3  ARKADIUSZ  GARDZIELEWSKI  WROCŁAW  POL   
2  3.0  3832  KRZYSZTOF          HADAS   POZNAŃ  POL   
3  4.0   416     DAMIAN         DYDUCH    KĘPNO  POL   

0     UKS BLIZA WŁADYSŁAWOWO  M  1  M30  ...  2.9266666666666667  00:44:47  \
1  ARKADIUSZGARDZIELEWSKI.PL  M  2  M30  ...  2.9833333333333334  00:45:26   
2                             M  3  M20  ...  3.1233333333333335  00:47:34   
3   AZS POLITECHNIKA OPOLSKA  M  4  M30  ...  3.1966666666666668  00:48:49   

0  1.0

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [5]:
# training_pipeline_fixed_v3.ipynb
import pandas as pd
import numpy as np
import boto3
import io
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
from dotenv import load_dotenv

# Załaduj zmienne środowiskowe
load_dotenv()

# Połączenie z DigitalOcean Spaces
session = boto3.session.Session()
s3 = session.client('s3',
                   region_name=os.getenv('DO_SPACES_REGION'),
                   endpoint_url=os.getenv('DO_SPACES_ENDPOINT'),
                   aws_access_key_id=os.getenv('DO_SPACES_KEY'),
                   aws_secret_access_key=os.getenv('DO_SPACES_SECRET'))

# Funkcja do wczytywania i przetwarzania każdego roku osobno
def process_year_data(filename, year):
    response = s3.get_object(Bucket=os.getenv('DO_SPACES_BUCKET'), Key=filename)
    df_temp = pd.read_csv(io.BytesIO(response['Body'].read()))
    
    # Rozdziel kolumny według średnika
    df_split = df_temp.iloc[:, 0].str.split(';', expand=True)
    
    # Ustaw nazwy kolumn z pierwszego wiersza
    original_columns = df_split.iloc[0].tolist()
    df_split.columns = original_columns
    df_split = df_split[1:]  # Usuń nagłówki
    
    # Mapowanie kolumn na podstawie analizy struktury
    column_mapping = {
        'Płeć': 'Płeć',
        'Rocznik': 'Rocznik', 
        '5 km Tempo': '5 km Tempo',
        'Czas': 'Czas'
    }
    
    # Znajdź właściwe kolumny
    actual_mapping = {}
    for target_col in column_mapping.keys():
        for actual_col in df_split.columns:
            if target_col in actual_col:
                actual_mapping[target_col] = actual_col
                break
    
    print(f"📊 {year} - znalezione kolumny:")
    for target, actual in actual_mapping.items():
        print(f"  {target}: {actual}")
    
    # Wybierz tylko potrzebne kolumny
    selected_data = df_split[list(actual_mapping.values())].copy()
    selected_data.columns = list(actual_mapping.keys())  # Ustandaryzuj nazwy
    selected_data['Year'] = year
    
    return selected_data

print("📥 Przetwarzanie danych 2023...")
df_2023 = process_year_data('halfmarathon_wroclaw_2023__final.csv', 2023)

print("\n📥 Przetwarzanie danych 2024...")
df_2024 = process_year_data('halfmarathon_wroclaw_2024__final.csv', 2024)

# Połącz dane
df = pd.concat([df_2023, df_2024], ignore_index=True)
print(f"\n📊 Połączone dane: {df.shape}")

print("\n🔍 PRZYKŁADOWE DANE:")
print(df.head())

# Przygotowanie danych do modelu
print("\n🔧 Przygotowanie danych do trenowania...")

# Funkcja do konwersji czasu na sekundy
def time_to_seconds(time_str):
    try:
        if pd.isna(time_str) or time_str == '':
            return None
        time_str = str(time_str).strip()
        parts = time_str.split(':')
        if len(parts) == 3:  # HH:MM:SS
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        elif len(parts) == 2:  # MM:SS
            return int(parts[0]) * 60 + int(parts[1])
        else:
            return None
    except:
        return None

# Funkcja do konwersji tempa (MM:SS/km) na sekundy/km
def pace_to_seconds(pace_str):
    try:
        if pd.isna(pace_str) or pace_str == '':
            return None
        pace_str = str(pace_str).strip()
        if '.' in pace_str:  # Liczba dziesiętna (minuty)
            return float(pace_str) * 60
        else:  # Format MM:SS
            parts = pace_str.split(':')
            if len(parts) == 2:
                return int(parts[0]) * 60 + int(parts[1])
            else:
                return None
    except:
        return None

# Konwersja danych
df['sex_numeric'] = df['Płeć'].map({'M': 1, 'K': 0, 'M ': 1, 'K ': 0})
df['Rocznik_num'] = pd.to_numeric(df['Rocznik'], errors='coerce')
df['half_marathon_seconds'] = df['Czas'].apply(time_to_seconds)
df['pace_5km_seconds'] = df['5 km Tempo'].apply(pace_to_seconds)

# Oblicz wiek
df['age'] = df['Year'] - df['Rocznik_num']

print(f"\n📊 Dane po przetworzeniu:")
print(f"Przykładowe wiersze:")
sample_data = df[['Płeć', 'sex_numeric', 'Rocznik', 'age', '5 km Tempo', 'pace_5km_seconds', 'Czas', 'half_marathon_seconds']].head(10)
print(sample_data)

# Statystyki
print(f"\n📈 STATYSTYKI:")
print(f"Liczba rekordów: {len(df)}")
print(f"Płeć M/K: {df['sex_numeric'].value_counts().to_dict()}")
print(f"Wiek - min: {df['age'].min()}, max: {df['age'].max()}")
print(f"Czas półmaratonu - min: {df['half_marathon_seconds'].min()/60:.1f}min, max: {df['half_marathon_seconds'].max()/60:.1f}min")

# Przygotowanie cech i targetu
features = df[['sex_numeric', 'age', 'pace_5km_seconds']].dropna()
target = df.loc[features.index, 'half_marathon_seconds']

# Usuń rekordy gdzie target jest None
valid_idx = target.notna()
features = features[valid_idx]
target = target[valid_idx]

print(f"\n📊 Dane do trenowania:")
print(f"Cechy: {features.shape}")
print(f"Target: {target.shape}")

if len(features) > 100:
    # Podział na train/test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")
    
    # Trenowanie modelu
    print("\n🤖 Trenowanie modelu...")
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    
    # Ewaluacja
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"\n📊 Wyniki modelu:")
    print(f"MAE: {mae:.2f} sekund ({mae/60:.1f} minut)")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f} sekund")
    
    # Przykładowe predykcje
    print(f"\n🔍 Przykładowe predykcje:")
    for i in range(5):
        actual_min = y_test.iloc[i] / 60
        pred_min = y_pred[i] / 60
        diff = pred_min - actual_min
        print(f"  Rzeczywiste: {actual_min:.1f}min, Przewidziane: {pred_min:.1f}min (różnica: {diff:+.1f}min)")
    
    # Zapis modelu
    print("\n💾 Zapis modelu...")
    model_bytes = io.BytesIO()
    joblib.dump(model, model_bytes)
    model_bytes.seek(0)
    
    s3.upload_fileobj(model_bytes, 
                      os.getenv('DO_SPACES_BUCKET'), 
                      'trained_model.pkl')
    
    print("✅ Model zapisany w DO Spaces jako 'trained_model.pkl'!")
    
    # Ważność cech
    feature_importance = pd.DataFrame({
        'feature': ['Płeć', 'Wiek', 'Tempo_5km'],
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🎯 Ważność cech:")
    print(feature_importance)
    
else:
    print("❌ Za mało danych do trenowania!")

print("\n🎉 Pipeline zakończony!")

📥 Przetwarzanie danych 2023...
📊 2023 - znalezione kolumny:

📥 Przetwarzanie danych 2024...
📊 2024 - znalezione kolumny:

📊 Połączone dane: (21955, 1)

🔍 PRZYKŁADOWE DANE:
   Year
0  2023
1  2023
2  2023
3  2023
4  2023

🔧 Przygotowanie danych do trenowania...


KeyError: 'Płeć'

In [6]:
# training_pipeline_debug_columns.ipynb
import pandas as pd
import numpy as np
import boto3
import io
import os
from dotenv import load_dotenv

# Załaduj zmienne środowiskowe
load_dotenv()

# Połączenie z DigitalOcean Spaces
session = boto3.session.Session()
s3 = session.client('s3',
                   region_name=os.getenv('DO_SPACES_REGION'),
                   endpoint_url=os.getenv('DO_SPACES_ENDPOINT'),
                   aws_access_key_id=os.getenv('DO_SPACES_KEY'),
                   aws_secret_access_key=os.getenv('DO_SPACES_SECRET'))

def debug_columns(filename, year):
    print(f"\n🔍 DEBUG {year}:")
    response = s3.get_object(Bucket=os.getenv('DO_SPACES_BUCKET'), Key=filename)
    df_temp = pd.read_csv(io.BytesIO(response['Body'].read()))
    
    # Rozdziel kolumny według średnika
    df_split = df_temp.iloc[:, 0].str.split(';', expand=True)
    
    # Pokaż pierwsze 2 wiersze z numerami kolumn
    print("PIERWSZY WIERSZ (nagłówki):")
    for i, col in enumerate(df_split.iloc[0]):
        print(f"  Kolumna {i}: '{col}'")
    
    print("\nDRUGI WIERSZ (przykładowe dane):")
    for i, col in enumerate(df_split.iloc[1]):
        print(f"  Kolumna {i}: '{col}'")
    
    return df_split

print("📥 Debugowanie struktury danych...")
df_2023_debug = debug_columns('halfmarathon_wroclaw_2023__final.csv', 2023)
df_2024_debug = debug_columns('halfmarathon_wroclaw_2024__final.csv', 2024)

# Sprawdźmy które kolumny zawierają kluczowe informacje
print("\n🎯 SZUKAM KOLUMN Z DANymi:")
keywords = {
    'płeć': ['M', 'K'],
    'wiek': ['1990', '1995', '2000'], 
    'czas_5km': ['00:15:', '00:20:'],
    'czas_półmaraton': ['01:10:', '01:30:']
}

for year, df_debug in [('2023', df_2023_debug), ('2024', df_2024_debug)]:
    print(f"\n--- {year} ---")
    for i in range(min(10, len(df_debug.columns))):  # Sprawdź pierwsze 10 kolumn
        col_data = df_debug[i].iloc[1]  # Drugi wiersz (pierwsze dane)
        print(f"Kolumna {i}: '{col_data}'")

📥 Debugowanie struktury danych...

🔍 DEBUG 2023:
PIERWSZY WIERSZ (nagłówki):
  Kolumna 0: '1.0'
  Kolumna 1: '1787'
  Kolumna 2: 'TOMASZ'
  Kolumna 3: 'GRYCKO'
  Kolumna 4: ''
  Kolumna 5: 'POL'
  Kolumna 6: 'UKS BLIZA WŁADYSŁAWOWO'
  Kolumna 7: 'M'
  Kolumna 8: '1'
  Kolumna 9: 'M30'
  Kolumna 10: '1'
  Kolumna 11: '1992.0'
  Kolumna 12: '00:14:37'
  Kolumna 13: '1.0'
  Kolumna 14: '2.9233333333333333'
  Kolumna 15: '00:29:15'
  Kolumna 16: '1.0'
  Kolumna 17: '2.9266666666666667'
  Kolumna 18: '00:44:47'
  Kolumna 19: '1.0'
  Kolumna 20: '3.106666666666667'
  Kolumna 21: '01:01:43'
  Kolumna 22: '1.0'
  Kolumna 23: '3.3866666666666663'
  Kolumna 24: '0.031399999999999956'
  Kolumna 25: '01:04:59'
  Kolumna 26: '3.0805088093545074'

DRUGI WIERSZ (przykładowe dane):
  Kolumna 0: '2.0'
  Kolumna 1: '3'
  Kolumna 2: 'ARKADIUSZ'
  Kolumna 3: 'GARDZIELEWSKI'
  Kolumna 4: 'WROCŁAW'
  Kolumna 5: 'POL'
  Kolumna 6: 'ARKADIUSZGARDZIELEWSKI.PL'
  Kolumna 7: 'M'
  Kolumna 8: '2'
  Kolumna 9: 'M3

In [7]:
# training_pipeline_final.ipynb
import pandas as pd
import numpy as np
import boto3
import io
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
from dotenv import load_dotenv

# Załaduj zmienne środowiskowe
load_dotenv()

# Połączenie z DigitalOcean Spaces
session = boto3.session.Session()
s3 = session.client('s3',
                   region_name=os.getenv('DO_SPACES_REGION'),
                   endpoint_url=os.getenv('DO_SPACES_ENDPOINT'),
                   aws_access_key_id=os.getenv('DO_SPACES_KEY'),
                   aws_secret_access_key=os.getenv('DO_SPACES_SECRET'))

def process_year_data(filename, year):
    response = s3.get_object(Bucket=os.getenv('DO_SPACES_BUCKET'), Key=filename)
    df_temp = pd.read_csv(io.BytesIO(response['Body'].read()))
    
    # Rozdziel kolumny według średnika
    df_split = df_temp.iloc[:, 0].str.split(';', expand=True)
    
    # Mapowanie kolumn na podstawie debugu
    column_mapping = {
        'Płeć': 7,           # Kolumna z płcią (M/K)
        'Rocznik': 11,       # Kolumna z rocznikiem
        '5km_czas': 12,      # Kolumna z czasem na 5km
        'Czas_półmaraton': 25 # Kolumna z czasem półmaratonu
    }
    
    # Wybierz tylko potrzebne kolumny
    selected_data = pd.DataFrame()
    for col_name, col_idx in column_mapping.items():
        selected_data[col_name] = df_split[col_idx]
    
    # Usuń pierwszy wiersz (nagłówki)
    selected_data = selected_data.iloc[1:]
    selected_data['Year'] = year
    
    return selected_data

print("📥 Przetwarzanie danych 2023...")
df_2023 = process_year_data('halfmarathon_wroclaw_2023__final.csv', 2023)

print("📥 Przetwarzanie danych 2024...")
df_2024 = process_year_data('halfmarathon_wroclaw_2024__final.csv', 2024)

# Połącz dane
df = pd.concat([df_2023, df_2024], ignore_index=True)
print(f"\n📊 Połączone dane: {df.shape}")

print("\n🔍 PRZYKŁADOWE DANE:")
print(df.head())

# Przygotowanie danych do modelu
print("\n🔧 Przygotowanie danych do trenowania...")

# Funkcja do konwersji czasu HH:MM:SS na sekundy
def time_to_seconds(time_str):
    try:
        if pd.isna(time_str) or time_str == '':
            return None
        time_str = str(time_str).strip()
        parts = time_str.split(':')
        if len(parts) == 3:  # HH:MM:SS
            return int(parts[0]) * 3600 + int(parts[1]) * 60 + int(parts[2])
        elif len(parts) == 2:  # MM:SS
            return int(parts[0]) * 60 + int(parts[1])
        else:
            return None
    except:
        return None

# Konwersja danych
df['sex_numeric'] = df['Płeć'].map({'M': 1, 'K': 0})
df['Rocznik_num'] = pd.to_numeric(df['Rocznik'], errors='coerce')
df['half_marathon_seconds'] = df['Czas_półmaraton'].apply(time_to_seconds)
df['time_5km_seconds'] = df['5km_czas'].apply(time_to_seconds)

# Oblicz tempo na km z czasu 5km (sekundy/km)
df['pace_5km_seconds_per_km'] = df['time_5km_seconds'] / 5.0

# Oblicz wiek
df['age'] = df['Year'] - df['Rocznik_num']

print(f"\n📊 Dane po przetworzeniu:")
sample_cols = ['Płeć', 'sex_numeric', 'Rocznik', 'age', '5km_czas', 'time_5km_seconds', 'pace_5km_seconds_per_km', 'Czas_półmaraton', 'half_marathon_seconds']
print(df[sample_cols].head(10))

# Statystyki
print(f"\n📈 STATYSTYKI:")
print(f"Liczba rekordów: {len(df)}")
print(f"Płeć - M: {(df['sex_numeric'] == 1).sum()}, K: {(df['sex_numeric'] == 0).sum()}")
print(f"Wiek - min: {df['age'].min():.0f}, max: {df['age'].max():.0f}, średnia: {df['age'].mean():.1f}")
print(f"Czas półmaratonu - min: {df['half_marathon_seconds'].min()/60:.1f}min, max: {df['half_marathon_seconds'].max()/60:.1f}min")

# Przygotowanie cech i targetu
features = df[['sex_numeric', 'age', 'pace_5km_seconds_per_km']].copy()
target = df['half_marathon_seconds'].copy()

# Usuń brakujące wartości
mask = features.notna().all(axis=1) & target.notna()
features = features[mask]
target = target[mask]

print(f"\n📊 Dane do trenowania:")
print(f"Cechy: {features.shape}")
print(f"Target: {target.shape}")

if len(features) > 100:
    # Podział na train/test
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
    
    print(f"Train: {X_train.shape}, Test: {X_test.shape}")
    
    # Trenowanie modelu
    print("\n🤖 Trenowanie modelu...")
    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)
    
    # Ewaluacja
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    print(f"\n📊 Wyniki modelu:")
    print(f"MAE: {mae:.2f} sekund ({mae/60:.1f} minut)")
    print(f"RMSE: {rmse:.2f} sekund ({rmse/60:.1f} minut)")
    
    # Przykładowe predykcje
    print(f"\n🔍 Przykładowe predykcje (test set):")
    for i in range(5):
        actual_min = y_test.iloc[i] / 60
        pred_min = y_pred[i] / 60
        diff = pred_min - actual_min
        print(f"  Rzeczywiste: {actual_min:.1f}min, Przewidziane: {pred_min:.1f}min (różnica: {diff:+.1f}min)")
    
    # Zapis modelu
    print("\n💾 Zapis modelu...")
    model_bytes = io.BytesIO()
    joblib.dump(model, model_bytes)
    model_bytes.seek(0)
    
    s3.upload_fileobj(model_bytes, 
                      os.getenv('DO_SPACES_BUCKET'), 
                      'trained_model.pkl')
    
    print("✅ Model zapisany w DO Spaces jako 'trained_model.pkl'!")
    
    # Ważność cech
    feature_importance = pd.DataFrame({
        'feature': ['Płeć', 'Wiek', 'Tempo_5km'],
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\n🎯 Ważność cech:")
    print(feature_importance)
    
    print(f"\n🎉 Pipeline zakończony pomyślnie!")
    print(f"Model wytrenowany na {len(features)} rekordach")
    print(f"Średni błąd: {mae/60:.1f} minut")
    
else:
    print("❌ Za mało danych do trenowania!")

print("\n" + "="*50)
print("🎯 MODEL GOTOWY DO UŻYCIA W APLIKACJI!")
print("="*50)

📥 Przetwarzanie danych 2023...
📥 Przetwarzanie danych 2024...

📊 Połączone dane: (21955, 5)

🔍 PRZYKŁADOWE DANE:
  Płeć Rocznik  5km_czas Czas_półmaraton  Year
0    M  1986.0  00:14:48        01:06:23  2023
1    M  1996.0  00:15:46        01:08:24  2023
2    M  1988.0  00:16:11        01:10:16  2023
3    M  1995.0  00:16:12        01:10:27  2023
4    M  1983.0  00:16:09        01:10:34  2023

🔧 Przygotowanie danych do trenowania...

📊 Dane po przetworzeniu:
  Płeć  sex_numeric Rocznik   age  5km_czas  time_5km_seconds  \
0    M          1.0  1986.0  37.0  00:14:48             888.0   
1    M          1.0  1996.0  27.0  00:15:46             946.0   
2    M          1.0  1988.0  35.0  00:16:11             971.0   
3    M          1.0  1995.0  28.0  00:16:12             972.0   
4    M          1.0  1983.0  40.0  00:16:09             969.0   
5    M          1.0  1999.0  24.0  00:15:37             937.0   
6    M          1.0  1989.0  34.0  00:16:30             990.0   
7    M          1.